### Reading dosage and info

In [2]:
### command line input ###
# chr = as.numeric(args[1])
# k = as.numeric(args[2])
# gene_id = args[3]
### data import ###
options(stringsAsFactors=F)
library(glmnet)
library(foreach)
library(genio)




# read plink data all at once
#genotype_dir <- "/gpfs/scratch60/zhao/zy92/GTEX/cis_snp/"
genotype_dir <- "/ysm-gpfs/project/wl382/GTEx_v8/genotype/cis_loc/"
gene_idx <- 1
#gene_vec <- list.files(paste0(genotype_dir, chr_str))


ntune = 100
chr <- 2
chr_str <- paste0("chr", chr, "/")
gene_vec <- list.files(paste0(genotype_dir, chr_str))
gene_vec_cleaned <- as.character(sapply(gene_vec, function(x) unlist(strsplit(x, split = '\\.'))[1]))
gene_id = gene_vec[gene_idx]
gtex_dir = "/gpfs/scratch60/zhao/zy92/GTEX/" 
glist = list.files(paste0(gtex_dir, "adjusted_expr1/chr", chr))
g = glist[gene_idx]

# extract_genotype function
extract_genotype <- function(gene_idx, chr_str, genotype_dir, gene_vec) {
    gene_dir <- paste0(genotype_dir, chr_str, gene_vec[gene_idx], "/", gene_vec[gene_idx])
    genotype_info <- read_plink(gene_dir)
    
    print(paste0("INFO: genotype matrix dimension:", dim(genotype_info$X)[1], " * ", dim(genotype_info$X)[2]))
    return(genotype_info)
}

# extract genotypes and annotation tables
genotype_info <- extract_genotype(1, chr_str, genotype_dir, gene_vec)
genotype   <- genotype_info$X
bim <- genotype_info$bim
fam <- genotype_info$fam

# extract genotypes and annotation tables
genotype_info <- extract_genotype(gene_idx, chr_str, genotype_dir, gene_vec)
genotype   <- genotype_info$X
bim <- genotype_info$bim
fam <- genotype_info$fam

Reading: /ysm-gpfs/project/wl382/GTEx_v8/genotype/cis_loc/chr2/ENSG00000003137.8/ENSG00000003137.8.bim
Reading: /ysm-gpfs/project/wl382/GTEx_v8/genotype/cis_loc/chr2/ENSG00000003137.8/ENSG00000003137.8.fam
Reading: /ysm-gpfs/project/wl382/GTEx_v8/genotype/cis_loc/chr2/ENSG00000003137.8/ENSG00000003137.8.bed


[1] "INFO: genotype matrix dimension:6092 * 838"


Reading: /ysm-gpfs/project/wl382/GTEx_v8/genotype/cis_loc/chr2/ENSG00000003137.8/ENSG00000003137.8.bim
Reading: /ysm-gpfs/project/wl382/GTEx_v8/genotype/cis_loc/chr2/ENSG00000003137.8/ENSG00000003137.8.fam
Reading: /ysm-gpfs/project/wl382/GTEx_v8/genotype/cis_loc/chr2/ENSG00000003137.8/ENSG00000003137.8.bed


[1] "INFO: genotype matrix dimension:6092 * 838"


In [ ]:
### Genotype for a gene 


In [49]:
genotype[1:10, 1:10]

,GTEX-1117F,GTEX-111CU,GTEX-111FC,GTEX-111VG,GTEX-111YS,GTEX-1122O,GTEX-1128S,GTEX-113IC,GTEX-113JC,GTEX-117XS
chr2_71129368_C_T_b38,0,0,0,0,0,0,1,0,0,0
chr2_71129592_A_T_b38,0,0,0,1,0,0,0,0,0,0
chr2_71130065_C_A_b38,0,0,0,0,0,0,1,0,0,0
chr2_71130287_C_A_b38,2,1,0,0,0,0,2,2,2,0
chr2_71130375_C_T_b38,0,0,0,0,0,0,1,0,0,0
chr2_71130504_C_T_b38,2,1,0,0,0,0,2,2,2,0
chr2_71130580_G_C_b38,0,1,1,2,0,1,0,0,0,0
chr2_71130634_A_G_b38,0,0,0,0,0,0,0,0,0,0
chr2_71130975_A_G_b38,0,0,0,0,0,0,1,0,0,0
chr2_71131072_T_C_b38,0,0,0,0,0,0,1,0,0,0


In [47]:
head(gene_vec_cleaned)

[1] "ENSG00000003137" "ENSG00000003393" "ENSG00000003400" "ENSG00000003402"
[5] "ENSG00000003436" "ENSG00000003509"

In [32]:
head(fam)

fam,id,pat,mat,sex,pheno
<chr>,<chr>,<chr>,<chr>,<int>,<dbl>
GTEX-1117F,GTEX-1117F,0,0,0,-9
GTEX-111CU,GTEX-111CU,0,0,0,-9
GTEX-111FC,GTEX-111FC,0,0,0,-9
GTEX-111VG,GTEX-111VG,0,0,0,-9
GTEX-111YS,GTEX-111YS,0,0,0,-9
GTEX-1122O,GTEX-1122O,0,0,0,-9


In [37]:
length(gene_vec)
length(glist)

[1] 3866

[1] 1850

In [38]:
head(gene_vec)

[1] "ENSG00000003137.8"  "ENSG00000003393.14" "ENSG00000003400.14"
[4] "ENSG00000003402.19" "ENSG00000003436.15" "ENSG00000003509.15"

### Gene expression for different tissues

In [3]:
gtex_dir

[1] "/gpfs/scratch60/zhao/zy92/GTEX/"

In [14]:
### format for dose and info
# dose:
# GTEX-P4PP-0004-SM-2H2WW->GTEX-P4PP-0004-SM-2H2WW        DOSE    0.000   0.014   0.000   0.000   0.000
# info:
# SNP     REF(0)  ALT(1)  ALT_Frq MAF     AvgCall Rsq     Genotyped       LooRsq  EmpR    EmpRsq  Dose0   Dose1
# 1_157695412_G_A_b37     G       A       -               -       -       Typed_Only      -       -
#       -       -       -


Yt = dir(paste0(gtex_dir, "adjusted_expr1/chr", chr, "/", g, "/"))
P = length(Yt)

output_dir = "/ysm-gpfs/pi/zhao/zy92/projects/utmost_update/GTEX/utmost_models/" 

dir.create(paste0(output_dir, "chr", chr), showWarnings = FALSE)
dir.create(paste0(output_dir, "chr", chr, "/", gene_id), showWarnings = FALSE)
setwd(paste0(output_dir, "chr", chr, "/", gene_id))
## expr files ##
Y = list()
for(t in 1:P){
    Y[[t]] = read.table(paste0(gtex_dir, "adjusted_expr/chr", chr, "/", g, "/", Yt[t]), header=F)
}
ssize = unlist(lapply(Y, nrow))
T_num = length(Yt)

[1] 49

### Genotype preprocess

In [15]:
# input genotype_info
dose = genotype_info$X
dose_std = dose

# standardizing the dosage matrix
for(j in 1:nrow(dose)){
    dose_std[j, is.na(dose[j, ])] <- mean(dose[j, ], na.rm = T)
    dose_std[j, ] <- dose_std[j, ] - mean(dose[j, ], na.rm = T)
}

## covariance matrix ##
n_sample = ncol(dose_std)
tmp = as.matrix(dose_std)

# genotype sample id 
sub_id = colnames(dose_std)
n_snp = nrow(dose_std)
id_tissue_map = list()
id_tissue_map_nn = list()

# iteration
for(t in 1:T_num){
    exp_id <- as.character(sapply(Y[[t]][,1], function(x) substr(x, 1, 10)))

    # index of ids that have matched genotypes 
    # exp_id based on order of exp_id
    id_tissue_map[[t]] <- match(sub_id, exp_id)
    id_tissue_map_nn[[t]] <- !is.na(id_tissue_map[[t]])
    id_tissue_map[[t]] <- na.omit(match(sub_id, exp_id))
    #print(length(id_tissue_map[[t]]))
    
}

In [31]:
X = list()
Yt = list()
lm_df = c()
cis_snp_vec = c()

for(t in 1:T_num){
    # filtering data
    IRdisplay::display_html(paste0("INFO: tissue", t))
    X[[t]] = apply(as.matrix(dose_std[,id_tissue_map_nn[[t]]]), 1, as.numeric)
    Yt[[t]] = Y[[t]][id_tissue_map[[t]], 2]
    if (length(lm_df) == 0) {
        lm_df = data.frame(cbind(X[[t]], Yt[[t]]))
    } else { 
        lm_df = rbind(lm_df, cbind(X[[t]], Yt[[t]]))
    }
    
}
cis_snp_vec = colnames(lm_df)[1:(ncol(lm_df)-1)]
colnames(lm_df)[ncol(lm_df)] <- "y"

INFO: tissue1

INFO: tissue2

INFO: tissue3

INFO: tissue4

INFO: tissue5

...

INFO: tissue45

INFO: tissue46

INFO: tissue47

INFO: tissue48

INFO: tissue49

In [37]:
colnames(lm_df)[ncol(lm_df)] <- "y"

In [38]:
colnames(lm_df)[ncol(lm_df)]

[1] "y"

In [32]:
dim(lm_df)

[1] 10254  6093

In [39]:
mod_multi = lm(y~., data = lm_df)

In [ ]:
system

In [28]:
t = 1
lm_df = data.frame(cbind(X[[t]], Yt[[t]]))
dim(lm_df)

[1]  376 6093

In [29]:
colnames(lm_df)

[1] "chr2_71129368_C_T_b38"                                                                                         
   [2] "chr2_71129592_A_T_b38"                                                                                         
   [3] "chr2_71130065_C_A_b38"                                                                                         
   [4] "chr2_71130287_C_A_b38"                                                                                         
   [5] "chr2_71130375_C_T_b38"                                                                                         
   [6] "chr2_71130504_C_T_b38"                                                                                         
   [7] "chr2_71130580_G_C_b38"                                                                                         
   [8] "chr2_71130634_A_G_b38"                                                                                         
   [9] "chr2_71130975_A_G_b38"                                                                                         
  [10] "chr2_71131072_T_C_b38"                                                                                         
  [11] "chr2_71131131_C_T_b38"                                                                                         
  [12] "chr2_71131515_T_C_b38"                                                                                         
  [13] "chr2_71131552_C_A_b38"                                                                                         
  [14] "chr2_71131579_A_G_b38"                                                                                         
  [15] "chr2_71131607_A_G_b38"                                                                                         
  [16] "chr2_71131875_C_T_b38"                                                                                         
  [17] "chr2_71132131_A_G_b38"                                                                                         
  [18] "chr2_71132480_A_G_b38"                                                                                         
  [19] "chr2_71132575_A_G_b38"                                                                                         
  [20] "chr2_71132702_G_A_b38"                                                                                         
  [21] "chr2_71132970_G_A_b38"                                                                                         
  [22] "chr2_71133014_A_C_b38"                                                                                         
  [23] "chr2_71133152_G_A_b38"                                                                                         
  [24] "chr2_71133495_G_T_b38"                                                                                         
  [25] "chr2_71133593_A_G_b38"                                                                                         
  [26] "chr2_71133605_T_TTCC_b38"                                                                                      
  [27] "chr2_71133774_T_C_b38"                                                                                         
  [28] "chr2_71133843_A_C_b38"                                                                                         
  [29] "chr2_71134202_T_G_b38"                                                                                         
  [30] "chr2_71134488_T_G_b38"                                                                                         
  [31] "chr2_71134916_T_A_b38"                                                                                         
  [32] "chr2_71135339_A_C_b38"                                                                                         
  [33] "chr2_71135404_A_T_b38"                                                                                         
  [34] "chr2_71135526_G_T_b38"             

In [27]:
length(lm_df)

[1] 6093

In [23]:
dim(X[[1]])
class(X[[1]])
length(Yt[[1]])
class(Yt[[1]])

[1]  376 6092

[1] "matrix"

[1] 376

[1] "numeric"

In [24]:
dim(X[[2]])
class(X[[2]])
length(Yt[[2]])
class(Yt[[2]])

[1]  365 6092

[1] "matrix"

[1] 365

[1] "numeric"

In [ ]:
lm()

### correlation calculation

In [ ]:
X_test = list()
Y_test = list()
X_tune = list()
Y_tune = list()
X_train = list()
Y_train = list()
for(t in 1:T_num){
    X_train_tmp = sub_id_map[[t]][!(sub_id_map[[t]]%in%c(test_index,tuning_index))]
    Y_train_tmp = !(sub_id_map[[t]]%in%c(test_index,tuning_index))
    X_tuning_tmp = sub_id_map[[t]][(sub_id_map[[t]]%in%tuning_index)]
    Y_tuning_tmp = (sub_id_map[[t]]%in%tuning_index)
    X_test_tmp = sub_id_map[[t]][(sub_id_map[[t]]%in%test_index)]
    Y_test_tmp = (sub_id_map[[t]]%in%test_index)
    # training data
    X_train[[t]] = apply(as.matrix(dose_std[,X_train_tmp]),1,as.numeric)
    Y_train[[t]] = Y[[t]][sub_id_map_genotype[[t]],][Y_train_tmp, 2]
    X_tune[[t]] = apply(as.matrix(dose_std[,X_tuning_tmp]),1,as.numeric)
    Y_tune[[t]] = Y[[t]][Y_tuning_tmp, 2]
    X_test[[t]] = apply(as.matrix(dose_std[,X_test_tmp]),1,as.numeric)
    Y_test[[t]] = Y[[t]][sub_id_map_genotype[[t]],][Y_test_tmp, 2]
}

## model training ##	
## train elastic net and used average lambda as tuning parameters ##
single_initial_est = matrix(0, ncol(X_train[[1]]), T_num)
single_summary = list()
for(t in 1:T_num){
    #print(Y_train[[t]])
    #print(sum(is.na(X_train[[t]])))
    tt = cv.glmnet(X_train[[t]], Y_train[[t]], alpha = 0.5, nfolds = 5)
    single_summary[[t]] = tt
    single_initial_est[,t] = tt$glmnet.fit$beta[,which.min(tt$cvm)]
}
## performance of Elastic net on tuning and testing data with various tuning parameters
els_output = elastic_net_mse(single_summary, X_tune, Y_tune, X_test, Y_test)

In [51]:
genotype_dir

[1] "/ysm-gpfs/project/wl382/GTEx_v8/genotype/cis_loc/"

In [24]:
paste0(gtex_dir, "adjusted_expr1/chr", chr)

[1] "/gpfs/scratch60/zhao/zy92/GTEX/adjusted_expr1/chr2"

In [25]:
head(list.files(paste0(genotype_dir, chr_str)))

[1] "ENSG00000003137.8"  "ENSG00000003393.14" "ENSG00000003400.14"
[4] "ENSG00000003402.19" "ENSG00000003436.15" "ENSG00000003509.15"

In [28]:
length(Yt)
head(Yt)

[1] 49

[1] "Adipose_Subcutaneous.adj_expr"     "Adipose_Visceral_Omentum.adj_expr"
[3] "Adrenal_Gland.adj_expr"            "Artery_Aorta.adj_expr"            
[5] "Artery_Coronary.adj_expr"          "Artery_Tibial.adj_expr"

In [ ]:
genotype_info$X[1:6, 1:6]

In [ ]:
cv_config = cv_helper(450, fold)

In [ ]:
paste0(gtex_dir, "adjusted_expr1/chr", chr, "/", g, "/", Yt[t])

In [ ]:
sub_id_map[[t]]

In [58]:
genotype_info <- read_plink(dose_path)
    dose = genotype_info$X
    dose_std = dose
	for(j in 1:nrow(dose)){
		dose_std[j, is.na(dose[j, ])] <- mean(dose[j, ], na.rm = T)
        dose_std[j, ] <- dose_std[j, ] - mean(dose[j, ], na.rm = T)
	}
	print("INFO: section A")
	## covariance matrix ##
    n_sample = ncol(dose_std)
	tmp = as.matrix(dose_std)
	XX = t(tmp)%*%as.matrix(tmp)/n_sample
	Xnorm = diag(XX)
	remove(tmp)
    remove(XX)

    
	sub_id = colnames(dose_std)
	M = nrow(dose_std)
	sub_id_map = list()
    sub_id_map_genotype = list()
	for(t in 1:T_num){
    #for(t in 1:1){
		#tmp = rep(0, nrow(Y[[t]]))
        exp_id <- as.character(sapply(Y[[t]][,1], function(x) substr(x, 1, 10)))
        
        # index of ids that have matched genotypes 
        # exp_id based on order of exp_id
		sub_id_map[[t]] <- na.omit(match(exp_id, sub_id))
        sub_id_map_genotype[[t]] <- !is.na(match(exp_id, sub_id)) 
        print(length(sub_id_map_genotype[[t]]))
        #for(j in 1:length(tmp)){
            # extract the id starting GTEX-xxxxx
            
			#tmp[j] = which(sub_id==exp_id)
		#}
		#sub_id_map[[t]] = tmp
	}

Reading: /ysm-gpfs/project/wl382/GTEx_v8/genotype/cis_loc/chr2//ENSG00000003137.8/ENSG00000003137.8.bim
Reading: /ysm-gpfs/project/wl382/GTEx_v8/genotype/cis_loc/chr2//ENSG00000003137.8/ENSG00000003137.8.fam
Reading: /ysm-gpfs/project/wl382/GTEx_v8/genotype/cis_loc/chr2//ENSG00000003137.8/ENSG00000003137.8.bed


[1] "INFO: section A"
[1] 581
[1] 469
[1] 233
[1] 387
[1] 213
[1] 584
[1] 129
[1] 147
[1] 194
[1] 175
[1] 209
[1] 205
[1] 175
[1] 165
[1] 170
[1] 202
[1] 170
[1] 126
[1] 114
[1] 396
[1] 483
[1] 147
[1] 318
[1] 368
[1] 330
[1] 497
[1] 465
[1] 372
[1] 386
[1] 73
[1] 208
[1] 515
[1] 144
[1] 706
[1] 532
[1] 167
[1] 305
[1] 237
[1] 221
[1] 517
[1] 605
[1] 174
[1] 227
[1] 324
[1] 322
[1] 574
[1] 129
[1] 141
[1] 670


In [56]:
head(colnames(dose))

[1] "GTEX-1117F" "GTEX-111CU" "GTEX-111FC" "GTEX-111VG" "GTEX-111YS"
[6] "GTEX-1122O"

In [20]:
#gene_id = gene_vec
#if(P){
	
	
	## genotype files ##
	#dose = read.table(dose_path, header=F)
    genotype_info <- read_plink(dose_path)
    dose = genotype_info$X
    dose_std = dose
	for(j in 1:nrow(dose)){
		dose_std[j, is.na(dose[j, ])] <- mean(dose[j, ], na.rm = T)
        dose_std[j, ] <- dose_std[j, ] - mean(dose[j, ], na.rm = T)
	}
	print("INFO: section A")
	## covariance matrix ##
    n_sample = ncol(dose_std)
	tmp = as.matrix(dose_std)
	XX = t(tmp)%*%as.matrix(tmp)/n_sample
	Xnorm = diag(XX)
	remove(tmp)
    remove(XX)

    
	sub_id = colnames(dose_std)
	M = nrow(dose_std)
	sub_id_map = list()
    sub_id_map_genotype = list()
	for(t in 1:T_num){
    #for(t in 1:1){
		#tmp = rep(0, nrow(Y[[t]]))
        exp_id <- as.character(sapply(Y[[t]][,1], function(x) substr(x, 1, 10)))
        
        # index of ids that have matched genotypes 
        # exp_id based on order of exp_id
		sub_id_map[[t]] <- na.omit(match(exp_id, sub_id))
        sub_id_map_genotype[[t]] <- !is.na(match(exp_id, sub_id)) 
        print(length(sub_id_map_genotype[[t]]))
        #for(j in 1:length(tmp)){
            # extract the id starting GTEX-xxxxx
            
			#tmp[j] = which(sub_id==exp_id)
		#}
		#sub_id_map[[t]] = tmp
	}
                                      
	
                                      
                                      #
	fold = 5
	print("INFO: section B")
	cv_config = cv_helper(n_sample, fold)
	cv_perm = cv_config$perm
	cv_idx = cv_config$idx
	
	single_res_test = list()
	single_lam = matrix(0,5,P)
	single_theta_est = list()
	
	multi_res_test = list()
	multi_lam = rep(0,5)
	multi_theta_est = list()
	
	for(f in 1:fold){
		bgt = Sys.time()
        print(paste0("INFO: fold ", f))
		test_index = cv_perm[cv_idx[f,1]:cv_idx[f,2]]
		test_id = sub_id[test_index]
		tuning_index = cv_perm[cv_idx[f%%fold+1,1]:cv_idx[f%%fold+1,2]]
		tuning_id = sub_id[tuning_index]
        
		X_test = list()
		Y_test = list()
		X_tune = list()
		Y_tune = list()
		X_train = list()
		Y_train = list()
		for(t in 1:T_num){
			X_train_tmp = sub_id_map[[t]][!(sub_id_map[[t]]%in%c(test_index,tuning_index))]
			Y_train_tmp = !(sub_id_map[[t]]%in%c(test_index,tuning_index))
			X_tuning_tmp = sub_id_map[[t]][(sub_id_map[[t]]%in%tuning_index)]
			Y_tuning_tmp = (sub_id_map[[t]]%in%tuning_index)
			X_test_tmp = sub_id_map[[t]][(sub_id_map[[t]]%in%test_index)]
			Y_test_tmp = (sub_id_map[[t]]%in%test_index)
            # training data
			X_train[[t]] = apply(as.matrix(dose_std[,X_train_tmp]),1,as.numeric)
			Y_train[[t]] = Y[[t]][sub_id_map_genotype[[t]],][Y_train_tmp, 2]
			X_tune[[t]] = apply(as.matrix(dose_std[,X_tuning_tmp]),1,as.numeric)
			Y_tune[[t]] = Y[[t]][Y_tuning_tmp, 2]
			X_test[[t]] = apply(as.matrix(dose_std[,X_test_tmp]),1,as.numeric)
			Y_test[[t]] = Y[[t]][sub_id_map_genotype[[t]],][Y_test_tmp, 2]
		}
		
		## model training ##	
		## train elastic net and used average lambda as tuning parameters ##
		single_initial_est = matrix(0, ncol(X_train[[1]]), T_num)
		single_summary = list()
		for(t in 1:T_num){
            #print(Y_train[[t]])
            #print(sum(is.na(X_train[[t]])))
			tt = cv.glmnet(X_train[[t]], Y_train[[t]], alpha = 0.5, nfolds = 5)
			single_summary[[t]] = tt
			single_initial_est[,t] = tt$glmnet.fit$beta[,which.min(tt$cvm)]
		}
		## performance of Elastic net on tuning and testing data with various tuning parameters
		els_output = elastic_net_mse(single_summary, X_tune, Y_tune, X_test, Y_test)
		single_res_test[[f]] = els_output$mse
		single_lam[f,] = els_output$lam
		single_theta_est[[f]] = els_output$est
		remove(els_output)
		
		initial_numeric = as.numeric(single_initial_est)
		lam_range = minmax_lambda(single_summary)
		lam_V = seq(lam_range[1], lam_range[2], length.out = ntune)
		remove(single_summary); 
        remove(single_initial_est);
	
		## preparation
		XY = grad_prep(X_train, Y_train)
#		bgt = Sys.time()
		XX_train = lapply(X_train, function(x){t(x)%*%x/nrow(x)})
#		edt = Sys.time()
#		print(edt-bgt)
	
		
		res_tune = rep(0, ntune)
		best.lam = 0
	
#		bgt = Sys.time()
		for(lam in 1:ntune){
			single_est = matrix(initial_numeric, M, P)
			ans = glasso(X=X_train, Y=Y_train, X1=X_tune, Y1=Y_tune, XX=XX_train, XY=XY, Xnorm=Xnorm, lambda1=lam_V[lam], lambda2=lam_V[lam], theta=single_est)
			res_tune[lam] = ans$tune_err
			remove(single_est); remove(ans);
		}
#		edt = Sys.time()
#		print(edt-bgt)

		best.lam = lam_V[which.min(res_tune)]
		single_est = matrix(initial_numeric, M, P)
		ans = glasso(X=X_train, Y=Y_train, X1=X_tune, Y1=Y_tune, XX=XX_train, XY=XY, Xnorm=Xnorm, lambda1=best.lam, lambda2=best.lam, theta=single_est)
		multi_res_test[[f]] = multi_mse(ans$est, X_test, Y_test)
		multi_lam[f] = best.lam
		multi_theta_est[[f]] = ans$est
		remove(single_est)
        remove(ans)
		edt = Sys.time()
		print(edt-bgt)
	}
    print("INFO: section C")
	save(single_res_test, single_lam, single_theta_est, multi_res_test, multi_lam, multi_theta_est, file = paste0('chr', chr, '.', k, '.', gene_id, ".RData"))
	
    # evaluation                                  
    res_single = avg_perm(single_res_test)
	res_multi = avg_perm(multi_res_test)
	cat("Elastic net average testing error (all): ", apply(res_single, 2, mean), '\n')
    cat("glasso averge testing error (all): ", apply(res_multi, 2, mean), '\n')
	cat("Number of all zero tissues in elastic net is ", sum(is.na(res_single[,1])), '\n')
	cat("Number of all zero tissues in glasso is ", sum(is.na(res_multi[,1])), '\n')
	#cat("Elastic net average testing error (non-zero): ", apply(res_single[!is.na(res_multi[,1]),], 2, mean), '\n')
	#cat("glasso averge testing error (non-zero): ", apply(res_multi[!is.na(res_multi[,1]),], 2, mean), '\n')

	## generate an estimate with whole data ##
	X_all = list()
	Y_all = list()
	for(t in 1:T_num){
		X_all_tmp = sub_id_map[[t]]
		X_all[[t]] = apply(as.matrix(dose[X_all_tmp, ]), 2, as.numeric)
		Y_all[[t]] = Y[[t]][,2]
	}
	# initial values 
	single_initial_est = matrix(0, ncol(X_train[[1]]), T_num)
	for(t in 1:T_num){
		tt = cv.glmnet(X_all[[t]], Y_all[[t]], alpha = 0.5, nfolds = 5)
		single_initial_est[,t] = tt$glmnet.fit$beta[,which.min(tt$cvm)]
	}
	initial_numeric = as.numeric(single_initial_est)
	remove(single_initial_est)
	XY = grad_prep(X_all, Y_all)
	XX_all = lapply(X_all, function(x){t(x)%*%x/nrow(x)})
	tmp_res = rep(0, fold)
	for(f in 1:fold){
		ans = glasso_no_early_stopping(X=X_all, Y=Y_all, XX=XX_all, XY=XY, Xnorm=Xnorm, lambda1=multi_lam[f], lambda2=multi_lam[f], theta=matrix(initial_numeric,M,P))
		tmp_res[f] = ans$train_err
	}
	ans = glasso_no_early_stopping(X=X_all, Y=Y_all, XX=XX_all, XY=XY, Xnorm=Xnorm, lambda1=multi_lam[which.min(tmp_res)], lambda2=multi_lam[which.min(tmp_res)], theta=matrix(initial_numeric,M,P))
	# id ref alt
    info = genotype$bim[, c(2,5,6)] 
	downstream_est = data.frame(info[,1:3], ans$est)
    
	write.table(downstream_est, paste0(gene_id, ".est"), quote=F, row.names=F, col.names=c("SNP", "REF.0.", "ALT.1.", Yt))
#}

Reading: /gpfs/scratch60/fas/radev/zy92/GTEX/cis_snp/chr2/ENSG00000003137.8/ENSG00000003137.8.bim
Reading: /gpfs/scratch60/fas/radev/zy92/GTEX/cis_snp/chr2/ENSG00000003137.8/ENSG00000003137.8.fam
Reading: /gpfs/scratch60/fas/radev/zy92/GTEX/cis_snp/chr2/ENSG00000003137.8/ENSG00000003137.8.bed


[1] "INFO: section A"
[1] "GTEX-1117F" "GTEX-111CU" "GTEX-111FC" "GTEX-111VG" "GTEX-111YS"
[6] "GTEX-1122O"
[1] "GTEX-1117F" "GTEX-111CU" "GTEX-111YS" "GTEX-1122O" "GTEX-1128S"
[6] "GTEX-113JC"
[1] "GTEX-111CU" "GTEX-111YS" "GTEX-1122O" "GTEX-117YX" "GTEX-11DXX"
[6] "GTEX-11DXZ"
[1] "GTEX-111YS" "GTEX-1122O" "GTEX-1128S" "GTEX-117XS" "GTEX-117YW"
[6] "GTEX-11DXX"
[1] "GTEX-1117F" "GTEX-1122O" "GTEX-117YX" "GTEX-11DXX" "GTEX-11DXZ"
[6] "GTEX-11DYG"
[1] "GTEX-1117F" "GTEX-111FC" "GTEX-111YS" "GTEX-1122O" "GTEX-1128S"
[6] "GTEX-113IC"
[1] "GTEX-11NUK" "GTEX-11ONC" "GTEX-11PRG" "GTEX-11TTK" "GTEX-11ZU8"
[6] "GTEX-11ZVC"
[1] "GTEX-11DZ1" "GTEX-11GSO" "GTEX-11GSP" "GTEX-11UD1" "GTEX-11WQC"
[6] "GTEX-11WQK"
[1] "GTEX-1192X" "GTEX-11DYG" "GTEX-11DZ1" "GTEX-11GSP" "GTEX-11NUK"
[6] "GTEX-11NV4"
[1] "GTEX-11DYG" "GTEX-11DZ1" "GTEX-11EI6" "GTEX-11EMC" "GTEX-11GSP"
[6] "GTEX-11OF3"
[1] "GTEX-111FC" "GTEX-1128S" "GTEX-117XS" "GTEX-1192X" "GTEX-11DYG"
[6] "GTEX-11DZ1"
[1] "GTEX-1117F" "GTEX-111FC" "G

Warning message in Y_tune[[t]] - X_tune[[t]] %*% lst[[t]]$glmnet.fit$beta[, k]:
“longer object length is not a multiple of shorter object length”

ERROR: Error in mean((Y_tune[[t]] - X_tune[[t]] %*% lst[[t]]$glmnet.fit$beta[, : dims [product 70] do not match the length of object [106]


### Preparation

In [9]:
# prepa
dir.create(paste0(output_dir, "chr", chr), showWarnings = FALSE)
dir.create(paste0(output_dir, "chr", chr, "/", gene_id), showWarnings = FALSE)
setwd(paste0(output_dir, "chr", chr, "/", gene_id))
## expr files ##
Y = list()
for(t in 1:P){
    Y[[t]] = read.table(paste0(gtex_dir, "adjusted_expr1/chr", chr, "/", g, "/", Yt[t]), header=F)
}
ssize = unlist(lapply(Y, nrow))
T_num = length(Yt)

## genotype files ##
#dose = read.table(dose_path, header=F)
genotype_info <- read_plink(dose_path)
dose = genotype_info$X
dose_std = dose
for(j in 1:nrow(dose)){
    dose_std[j, is.na(dose[j, ])] <- mean(dose[j, ], na.rm = T)
    dose_std[j, ] <- dose_std[j, ] - mean(dose[j, ], na.rm = T)
}
print("INFO: section A")
## covariance matrix ##
n_sample = ncol(dose_std)
tmp = as.matrix(dose_std)
XX = t(tmp)%*%as.matrix(tmp)/n_sample
Xnorm = diag(XX)
remove(tmp)
remove(XX)
sub_id = colnames(dose_std)
M = nrow(dose_std)
sub_id_map = list()
sub_id_map_genotype = list()
for(t in 1:T_num){
#for(t in 1:1){
    #tmp = rep(0, nrow(Y[[t]]))
    exp_id <- as.character(sapply(Y[[t]][,1], function(x) substr(x, 1, 10)))
    #print(head(exp_id))
    # index of ids that have matched genotypes 
    # exp_id based on order of exp_id
    sub_id_map[[t]] <- na.omit(match(exp_id, sub_id))
    sub_id_map_genotype[[t]] <- !is.na(match(exp_id, sub_id))                           
    #for(j in 1:length(tmp)){
        # extract the id starting GTEX-xxxxx

        #tmp[j] = which(sub_id==exp_id)
    #}
    #sub_id_map[[t]] = tmp
}

fold = 5
print("INFO: section B")
cv_config = cv_helper(n_sample, fold)
cv_perm = cv_config$perm
cv_idx = cv_config$idx

single_res_test = list()
single_lam = matrix(0,5,P)
single_theta_est = list()

multi_res_test = list()
multi_lam = rep(0,5)
multi_theta_est = list()

Reading: /gpfs/scratch60/fas/radev/zy92/GTEX/cis_snp/chr2/ENSG00000003137.8/ENSG00000003137.8.bim
Reading: /gpfs/scratch60/fas/radev/zy92/GTEX/cis_snp/chr2/ENSG00000003137.8/ENSG00000003137.8.fam
Reading: /gpfs/scratch60/fas/radev/zy92/GTEX/cis_snp/chr2/ENSG00000003137.8/ENSG00000003137.8.bed


[1] "INFO: section A"
[1] "INFO: section B"


### CV

In [19]:
### fold = 1
    f = 1
    bgt = Sys.time()
    IRdisplay::display_html(paste0("INFO: fold ", f))
    test_index = cv_perm[cv_idx[f,1]:cv_idx[f,2]]
    test_id = sub_id[test_index]
    tuning_index = cv_perm[cv_idx[f%%fold+1,1]:cv_idx[f%%fold+1,2]]
    tuning_id = sub_id[tuning_index]

    X_test = list()
    Y_test = list()
    X_tune = list()
    Y_tune = list()
    X_train = list()
    Y_train = list()
    for(t in 1:T_num){
        X_train_tmp = sub_id_map[[t]][!(sub_id_map[[t]]%in%c(test_index,tuning_index))]
        Y_train_tmp = !(sub_id_map[[t]]%in%c(test_index,tuning_index))
        X_tuning_tmp = sub_id_map[[t]][(sub_id_map[[t]]%in%tuning_index)]
        Y_tuning_tmp = (sub_id_map[[t]]%in%tuning_index)
        X_test_tmp = sub_id_map[[t]][(sub_id_map[[t]]%in%test_index)]
        Y_test_tmp = (sub_id_map[[t]]%in%test_index)
        # training data
        X_train[[t]] = apply(as.matrix(dose_std[,X_train_tmp]),1,as.numeric)
        Y_train[[t]] = Y[[t]][sub_id_map_genotype[[t]],][Y_train_tmp, 2]
        X_tune[[t]] = apply(as.matrix(dose_std[,X_tuning_tmp]),1,as.numeric)
        Y_tune[[t]] = Y[[t]][sub_id_map_genotype[[t]],][Y_tuning_tmp, 2]
        X_test[[t]] = apply(as.matrix(dose_std[,X_test_tmp]),1,as.numeric)
        Y_test[[t]] = Y[[t]][sub_id_map_genotype[[t]],][Y_test_tmp, 2]
    }

    ## model training ##	
    ## train elastic net and used average lambda as tuning parameters ##
    single_initial_est = matrix(0, ncol(X_train[[1]]), T_num)
    single_summary = list()
    for(t in 1:T_num){
        IRdisplay::display_html(length(Y_train[[t]]))
        IRdisplay::display_html(dim(X_train[[t]]))
        tt = cv.glmnet(X_train[[t]], Y_train[[t]], alpha = 0.5, nfolds = 5)
        single_summary[[t]] = tt
        single_initial_est[,t] = tt$glmnet.fit$beta[,which.min(tt$cvm)]
    }
    ## performance of Elastic net on tuning and testing data with various tuning parameters
    els_output = elastic_net_mse(single_summary, X_tune, Y_tune, X_test, Y_test)
    single_res_test[[f]] = els_output$mse
    single_lam[f,] = els_output$lam
    single_theta_est[[f]] = els_output$est
    remove(els_output)

    initial_numeric = as.numeric(single_initial_est)
    lam_range = minmax_lambda(single_summary)
    lam_V = seq(lam_range[1], lam_range[2], length.out = ntune)
    remove(single_summary); 
    remove(single_initial_est);

    ## preparation
    XY = grad_prep(X_train, Y_train)
#		bgt = Sys.time()
    XX_train = lapply(X_train, function(x){t(x)%*%x/nrow(x)})
#		edt = Sys.time()
#		print(edt-bgt)


    res_tune = rep(0, ntune)
    best.lam = 0

#		bgt = Sys.time()
    print("INFO: section C")
    for(lam in 1:ntune){
        single_est = matrix(initial_numeric, M, P)
        ans = glasso(X=X_train, Y=Y_train, X1=X_tune, Y1=Y_tune, XX=XX_train, XY=XY, Xnorm=Xnorm, lambda1=lam_V[lam], lambda2=lam_V[lam], theta=single_est)
        res_tune[lam] = ans$tune_err
        remove(single_est); remove(ans);
    }
#		edt = Sys.time()
#		print(edt-bgt)
    
    
    best.lam = lam_V[which.min(res_tune)]
    single_est = matrix(initial_numeric, M, P)
    ans = glasso(X=X_train, Y=Y_train, X1=X_tune, Y1=Y_tune, XX=XX_train, XY=XY, Xnorm=Xnorm, lambda1=best.lam, lambda2=best.lam, theta=single_est)
    multi_res_test[[f]] = multi_mse(ans$est, X_test, Y_test)
    multi_lam[f] = best.lam
    multi_theta_est[[f]] = ans$est
    remove(single_est)
    remove(ans)
    edt = Sys.time()
    print(edt-bgt)

INFO: fold 1

[1] "INFO: section C"


Training error: 763.26731292237

Testing error: 989.543517344097

Training error: NaN

Testing error: NaN

ERROR: Error in if (new_objV2 > old_objV2 | new_objV1 > old_objV1) {: missing value where TRUE/FALSE needed


In [12]:
for(f in 1:fold){
    bgt = Sys.time()
    IRdisplay::display_html(paste0("INFO: fold ", f))
    test_index = cv_perm[cv_idx[f,1]:cv_idx[f,2]]
    test_id = sub_id[test_index]
    tuning_index = cv_perm[cv_idx[f%%fold+1,1]:cv_idx[f%%fold+1,2]]
    tuning_id = sub_id[tuning_index]

    X_test = list()
    Y_test = list()
    X_tune = list()
    Y_tune = list()
    X_train = list()
    Y_train = list()
    for(t in 1:T_num){
        X_train_tmp = sub_id_map[[t]][!(sub_id_map[[t]]%in%c(test_index,tuning_index))]
        Y_train_tmp = !(sub_id_map[[t]]%in%c(test_index,tuning_index))
        X_tuning_tmp = sub_id_map[[t]][(sub_id_map[[t]]%in%tuning_index)]
        Y_tuning_tmp = (sub_id_map[[t]]%in%tuning_index)
        X_test_tmp = sub_id_map[[t]][(sub_id_map[[t]]%in%test_index)]
        Y_test_tmp = (sub_id_map[[t]]%in%test_index)
        # training data
        X_train[[t]] = apply(as.matrix(dose_std[,X_train_tmp]),1,as.numeric)
        Y_train[[t]] = Y[[t]][sub_id_map_genotype[[t]],][Y_train_tmp, 2]
        X_tune[[t]] = apply(as.matrix(dose_std[,X_tuning_tmp]),1,as.numeric)
        Y_tune[[t]] = Y[[t]][sub_id_map_genotype[[t]],][Y_tuning_tmp, 2]
        X_test[[t]] = apply(as.matrix(dose_std[,X_test_tmp]),1,as.numeric)
        Y_test[[t]] = Y[[t]][sub_id_map_genotype[[t]],][Y_test_tmp, 2]
    }

    ## model training ##	
    ## train elastic net and used average lambda as tuning parameters ##
    single_initial_est = matrix(0, ncol(X_train[[1]]), T_num)
    single_summary = list()
    for(t in 1:T_num){
        IRdisplay::display_html(length(Y_train[[t]]))
        IRdisplay::display_html(dim(X_train[[t]]))
        tt = cv.glmnet(X_train[[t]], Y_train[[t]], alpha = 0.5, nfolds = 5)
        single_summary[[t]] = tt
        single_initial_est[,t] = tt$glmnet.fit$beta[,which.min(tt$cvm)]
    }
    ## performance of Elastic net on tuning and testing data with various tuning parameters
    els_output = elastic_net_mse(single_summary, X_tune, Y_tune, X_test, Y_test)
    single_res_test[[f]] = els_output$mse
    single_lam[f,] = els_output$lam
    single_theta_est[[f]] = els_output$est
    remove(els_output)

    initial_numeric = as.numeric(single_initial_est)
    lam_range = minmax_lambda(single_summary)
    lam_V = seq(lam_range[1], lam_range[2], length.out = ntune)
    remove(single_summary); 
    remove(single_initial_est);

    ## preparation
    XY = grad_prep(X_train, Y_train)
#		bgt = Sys.time()
    XX_train = lapply(X_train, function(x){t(x)%*%x/nrow(x)})
#		edt = Sys.time()
#		print(edt-bgt)


    res_tune = rep(0, ntune)
    best.lam = 0

#		bgt = Sys.time()
    print("INFO: section C")
    for(lam in 1:ntune){
        single_est = matrix(initial_numeric, M, P)
        ans = glasso(X=X_train, Y=Y_train, X1=X_tune, Y1=Y_tune, XX=XX_train, XY=XY, Xnorm=Xnorm, lambda1=lam_V[lam], lambda2=lam_V[lam], theta=single_est)
        res_tune[lam] = ans$tune_err
        remove(single_est); remove(ans);
    }
#		edt = Sys.time()
#		print(edt-bgt)
    
    
    best.lam = lam_V[which.min(res_tune)]
    single_est = matrix(initial_numeric, M, P)
    ans = glasso(X=X_train, Y=Y_train, X1=X_tune, Y1=Y_tune, XX=XX_train, XY=XY, Xnorm=Xnorm, lambda1=best.lam, lambda2=best.lam, theta=single_est)
    multi_res_test[[f]] = multi_mse(ans$est, X_test, Y_test)
    multi_lam[f] = best.lam
    multi_theta_est[[f]] = ans$est
    remove(single_est)
    remove(ans)
    edt = Sys.time()
    print(edt-bgt)
}

INFO: fold 1

In [24]:
    t <- 1
dim(X_train[[t]])
length(Y_train[[t]])
head(Y_train[[t]])

[1]  228 6092

[1] 352

[1] -1.0074211 -0.4055381  1.1721721 -4.6879187 -2.7419296  1.6128966

In [26]:
length(Y[[t]][Y_train_tmp, 2])

[1] 352

In [31]:
head(Y_train_tmp)
sum(Y_train_tmp)

[1] FALSE  TRUE  TRUE FALSE FALSE FALSE

[1] 228

In [32]:
length(Y_train_tmp)

[1] 376

In [12]:
t = 1
X_train_tmp = sub_id_map[[t]][!(sub_id_map[[t]]%in%c(test_index,tuning_index))]
Y_train_tmp = !(sub_id_map[[t]]%in%c(test_index,tuning_index))

In [16]:
length(sub_id_map[[1]])

[1] 376

In [17]:
length(X_train_tmp)
sum(Y_train_tmp)

[1] 228

[1] 228

In [8]:
X_train[[t]][1:6, 1:6]

chr2_71129368_C_T_b38,chr2_71129592_A_T_b38,chr2_71130065_C_A_b38,chr2_71130287_C_A_b38,chr2_71130375_C_T_b38,chr2_71130504_C_T_b38
-0.0274463,-0.3639618,-0.0274463,0.2875895,-0.0274463,0.2899761
-0.0274463,-0.3639618,-0.0274463,-0.7124105,-0.0274463,-0.7100239
0.9725537,-0.3639618,0.9725537,1.2875895,0.9725537,1.2899761
-0.0274463,0.6360382,-0.0274463,0.2875895,-0.0274463,0.2899761
-0.0274463,-0.3639618,-0.0274463,0.2875895,-0.0274463,0.2899761
-0.0274463,0.6360382,-0.0274463,0.2875895,-0.0274463,0.2899761


In [80]:
sum(is.na(X_train[[t]]))

[1] 1248860

In [73]:
sum(is.na(dose))
sum(is.na(dose_std))
dim(dose)
dim(dose_std)

[1] 3147

[1] 0

[1] 6092  838

[1] 6092  838

In [21]:
dim(X_train[[t]])
length(Y_train[[t]])

[1]  228 6092

[1] 352

In [75]:
class(X_train_tmp)

[1] "integer"

In [82]:
exp_id <- as.character(sapply(Y[[t]][,1], function(x) substr(x, 1, 10)))
print(head(exp_id))
sub_id_map[[t]] <- match(exp_id, sub_id)

[1] "GTEX-1117F" "GTEX-111CU" "GTEX-111FC" "GTEX-111VG" "GTEX-111YS"
[6] "GTEX-1122O"


In [84]:
sum(is.na(sub_id_map[[t]]))
length(sub_id_map[[t]])

[1] 205

[1] 581

In [85]:
length(sub_id)

[1] 838

In [ ]:
for(t in 1:T_num){
    #for(t in 1:1){
		#tmp = rep(0, nrow(Y[[t]]))
        exp_id <- as.character(sapply(Y[[t]][,1], function(x) substr(x, 1, 10)))
        print(head(exp_id))
		sub_id_map[[t]] <- match(exp_id, sub_id)
        #for(j in 1:length(tmp)){
            # extract the id starting GTEX-xxxxx
            
			#tmp[j] = which(sub_id==exp_id)
		#}
		#sub_id_map[[t]] = tmp
	}